In [1]:
!pip install wandb


In [2]:
from __future__ import print_function, division
import wandb
import sys
sys.path.append('../src/')
sys.path.append('..')

import src.utils as my_utils
from train import *
from prepare_data import *

from dataclasses import dataclass, replace
from pathlib import Path
from typing import Callable, List, Optional, Sequence, Union

import pandas as pd
import torch 
import random
import os
from pandas import DataFrame
from PIL import Image
from torch.utils.data.dataloader import default_collate
import sklearn
import shutil
from sklearn.metrics import f1_score, roc_auc_score, precision_score

import numpy as np
from matplotlib import pyplot as plt

import numpy as np
from torchvision import transforms

from torchvision import models
from torch.nn.modules import loss
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm.auto import tqdm

import os
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from torchvision.transforms import RandomVerticalFlip, RandomCrop

from albumentations import RandomCrop, Normalize, HorizontalFlip, Resize
from albumentations import Compose
from albumentations.pytorch import ToTensor

import datetime
from autoencoder import Autoencoder
from train import *
from prepare_data import *
from model import *
from utils import *

In [3]:
LR = 1e-3
BATCH_SIZE = 64
IMAGE_SIZE = 224

PATH = '/project/'
LOG_PATH = PATH +'logs/'
DATA_PATH = PATH + 'data/'
DEVICE = torch.device("cuda")
EPOCHS  = 200
#NAME = args.tags + '_'+ BASE + '_' + TYPE + '_' + str(LR) + '_' + str(NUM_CLASSES)
NAME = 'hyper_tune'    
DEVICE = torch.device("cuda")
NUM_CLASSES = 6

In [4]:
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.cuda.manual_seed(42)

torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [5]:
my_transform = strong_aug(p=0.5)
train_loader, test_loader = prepare_dataset(csv_file_uf=DATA_PATH+'data_uf.csv',csv_file_dc=DATA_PATH+'data_dc.csv',
                                        root_dir=DATA_PATH, transform=my_transform, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, train_prop=0.8)
model, optimizer, loss = prepare_base_model(device = DEVICE, lr=1e-3)
train_len, train_loss = train(model, DEVICE, train_loader, optimizer, loss, 0, name = NAME, num_classes=NUM_CLASSES, log_path = LOG_PATH)
    

0


KeyboardInterrupt: 

In [6]:
def run_model(*params):
    my_transform = strong_aug(p=0.5)
    params = (params[0])
    print('*************************')
    print(f"bs:  {params['batch_size']}, lr: {params['lr']}, b1: {params['beta_1']}, b2: {params['beta_2']}, weight decay: {params['weight_decay']} ")
    train_loader, test_loader = prepare_dataset(csv_file_uf=DATA_PATH+'data_uf.csv',csv_file_dc=DATA_PATH+'data_dc.csv',
                                        root_dir=DATA_PATH, transform=my_transform, image_size=IMAGE_SIZE, batch_size=params['batch_size'], train_prop=0.8)

    model, optimizer, loss = prepare_base_model(device = DEVICE, lr=params['lr'], beta_1=params['beta_1'],\
                                                beta_2=params['beta_2'], weight_decay=params['weight_decay'])
    for epoch in range(2):
        train_len, train_loss = train(model, DEVICE, train_loader, optimizer, loss, epoch, name = NAME, num_classes=NUM_CLASSES, log_path = LOG_PATH, wandb_log=0)
    return train_loss


In [6]:
!pip install hyperopt

In [8]:

import numpy as np
from hyperopt import hp, tpe, fmin

space =  {'batch_size' : hp.choice('batch_size', range(2, 27, 2)),
          'lr': hp.uniform('lr', 1e-2, 1e-6),
          'beta_1': hp.uniform('beat_1', 0.8,1),
          'beta_2': hp.uniform('beta_2', 0.9, 0.999),
          'weight_decay': hp.uniform('weight_decay', 0,1e-3)
        }

# Single line bayesian optimization of polynomial function
best = fmin(fn=lambda x: run_model(x),
            space=space,
            algo=tpe.suggest, 
            max_evals=20)
print(best)


bs:  16, lr: 0.00637138167769577, b1: 0.9915135095742895, b2: 0.9037234684041184, weight decay: 0.00023637019837627804 
0                                                   
  0%|          | 0/20 [00:00<?, ?it/s, best loss: ?]

Train Epoch: 0 	Loss: 0.086471    F1: 0.2690    My F1: 0.2690, AP: 0.2923
0                                                   
  0%|          | 0/20 [02:27<?, ?it/s, best loss: ?]

Train Epoch: 1 	Loss: 0.064105    F1: 0.2657    My F1: 0.2657, AP: 0.2943
bs:  2, lr: 0.0018837372831214897, b1: 0.8934982699114666, b2: 0.9842650406004902, weight decay: 0.0005618350136582198 
0                                                                                
  5%|▌         | 1/20 [04:57<1:34:07, 297.23s/it, best loss: 0.06410537790833858]

Train Epoch: 0 	Loss: 0.565753    F1: 0.1630    My F1: 0.1630, AP: 0.2080        
0                                                                                
  5%|▌         | 1/20 [07:54<1:34:07, 297.23s/it, best loss: 0.06410537790833858]

Train Epoch: 1 	Loss: 0.483728    F1: 0.1620    My F1: 0.1620, AP: 0.2011        
bs:  20, lr: 0.007104380101101872, b1: 0.9479314064995258, b2: 0.9344111854162556, weight decay: 0.000556571374153217 
0                                                                                
 10%|█         | 2/20 [10:46<1:33:48, 312.71s/it, best loss: 0.06410537790833858]

...

Train Epoch: 1 	Loss: 0.402301    F1: 0.2441    My F1: 0.2441, AP: 0.2175          
bs:  12, lr: 0.0043694362946720385, b1: 0.9965432283620652, b2: 0.9976553502909253, weight decay: 0.0004739331293649889 
0                                                                                  
 90%|█████████ | 18/20 [1:52:12<16:22, 491.07s/it, best loss: 0.031106401645881625]

Train Epoch: 0 	Loss: 0.082880    F1: 0.2460    My F1: 0.2460, AP: 0.2950          
0                                                                                  
 90%|█████████ | 18/20 [1:55:50<16:22, 491.07s/it, best loss: 0.031106401645881625]

Train Epoch: 1 	Loss: 0.067072    F1: 0.2607    My F1: 0.2607, AP: 0.3153          
bs:  10, lr: 0.007777553431482448, b1: 0.9014014770885906, b2: 0.9298029642924927, weight decay: 0.0008935283267363852 
0                                                                                  
 95%|█████████▌| 19/20 [1:59:27<07:54, 474.39s/it, best loss: 0.031106401645881625]

Train Epoch: 0 	Loss: 0.110082    F1: 0.2250    My F1: 0.2250, AP: 0.2394          
0                                                                                  
 95%|█████████▌| 19/20 [2:04:08<07:54, 474.39s/it, best loss: 0.031106401645881625]

Train Epoch: 1 	Loss: 0.082363    F1: 0.2488    My F1: 0.2488, AP: 0.2568          
100%|██████████| 20/20 [2:08:20<00:00, 492.18s/it, best loss: 0.031106401645881625]
{'batch_size': 12, 'beat_1': 0.8815615718640571, 'beta_2': 0.922051814583773, 'lr': 0.004843340424044938, 'weight_decay': 0.0002013364714059105}
